In [2]:
import tensorflow as tf
import data_utils
from qwk import quadratic_weighted_kappa
import time
import os
import sys
import pandas as pd
import numpy as np

graph = tf.get_default_graph()

essay_set_id = 7
num_tokens = 42
embedding_size = 300
num_samples = 1
is_regression = True

early_stop_count = 0
max_step_count = 10
is_regression = False
gated_addressing = False
# essay_set_id = 1
batch_size = 15
embedding_size = 300
feature_size = 100
l2_lambda = 0.3
hops = 3
reader = 'bow' # gru may not work
epochs = 200
num_samples = 1
num_tokens = 42
test_batch_size = batch_size
random_state = 0

training_path = 'training_set_rel3.tsv'
essay_list, resolved_scores, essay_id = data_utils.load_training_data(training_path, essay_set_id)
print(len(resolved_scores))

max_score = max(resolved_scores)
min_score = min(resolved_scores)
if essay_set_id == 7:
    min_score, max_score = 0, 30
elif essay_set_id == 8:
    min_score, max_score = 0, 60

print("Max Score", max_score)
print("Min Score", min_score)
score_range = range(min_score, max_score+1)
# load glove
word_idx, word2vec = data_utils.load_glove(num_tokens, dim=embedding_size)

vocab_size = len(word_idx) + 1
print("vocab size", vocab_size)
# stat info on data set

sent_size_list = list(map(len, [essay for essay in essay_list]))
max_sent_size = max(sent_size_list)
print("train max sent size: ", max_sent_size)
mean_sent_size = int(np.mean(sent_size_list))
E = data_utils.vectorize_data(essay_list, word_idx, max_sent_size)

percentage = ["five", "ten", "fifteen", "twenty"]
for percent in percentage:
    testcases_path = "percent/"+percent+"/p7/"
    # print(testcases_path)
    testcases_files = os.listdir(testcases_path)
    print("Number of files", len(testcases_files))
    testcases_files = sorted(testcases_files)

    # te
    # testcases_files.remove("babel_prompt7.csv")
    # testcases_files.remove("test7.csv")
    # testcases_files.remove("svo_triplets_all_prompt7.csv")
    # testcases_files.remove("svo_triplets_random_prompt7.csv")
    # testcases_files.remove("entities_beg_unbound_1.csv")
    # testcases_files.remove("entities_end_bound_1.csv")
    # testcases_files.remove("entities_end_unbound_1.csv")
    # testcases_files.remove("entities_mid_bound_1.csv")
    # testcases_files.remove("entities_mid_unbound_1.csv")
    print(testcases_files)

    # testcases_files = ['disfluency_1.csv', 'incorrect_grammar_1.csv']
    for testcase in testcases_files:
        print("Testcase: ", testcase)
        testing_path = testcases_path+testcase
        apple = True
    #     if testcase == 'incorrect_1.csv':
    #         df_essay = pd.read_csv(testing_path, engine='python')
    # #         df_essay = df_essay[:0]
    #         df_score = pd.read_csv('statistics/pred_prompt1.csv')
    #         df_essay['rating'] = df_score['pred_score']
    #         df_essay['text2'] = df_essay['text']
    #         df_essay = df_essay[['text', 'rating']]
    #         print(df_essay.head())
    #     if (testcase =='test8.csv' or 'babel_prompt1.csv'):
    #         continue
    #         df_essay = pd.read_csv(testing_path, engine='python')
    #         df_essay = df_essay.iloc[:,0]
    #         print(df_essay.head())
        if apple==True:
    #         print("Before: ", df_essay)
            df_essay = pd.read_csv(testing_path, engine='python')
    #         print("Before: ", df_essay)
    #         df_essay[:-1]
    #         print("After: ", df_essay)
            print(len(df_essay))
    #         df_score = pd.read_csv('predicted_scores/org_scores/prompt2_org.csv')
    #         print(len(df_score))
    #         df_essay['rating'] = df_score['score']
    #         print(df_essay.head())

    #     testing_path = 'aes_data/essay1/fold_0/test.txt'
        essay_list_test, essay_id_test = data_utils.load_testcase_data(df_essay, essay_set_id)

    #     sent_size_list_test = list(map(len, [essay2 for essay2 in essay_list_test]))
    #     max_sent_size_test = max(sent_size_list_test)
    #     print("test max sent size: ", max_sent_size_test)
    #     mean_sent_size_test = int(np.mean(sent_size_list_test))

        E_test = data_utils.vectorize_data(essay_list_test, word_idx, max_sent_size)
    #     print(essay_list_test)
        test_batch_size = 15

        testE = []
        test_scores = []
        test_essay_id = []
        for test_index in range(len(essay_id_test)):
            testE.append(E_test[test_index])
    #         print(testE)
    #         test_scores.append(resolved_scores_test[test_index])
            test_essay_id.append(essay_id_test[test_index])

        trainE = []
        train_scores = []
        train_essay_id = []
        for train_index in range(len(essay_id)):
            trainE.append(E[train_index])
            train_scores.append(resolved_scores[train_index])
            train_essay_id.append(essay_id[train_index])

        n_train = len(trainE)    
        n_test = len(testE)
        print("training_data: ", n_train)
        print("Testcase: ", testcase, n_test)

        memory = []
        for i in score_range:
            for j in range(num_samples):
                if i in train_scores:
                    score_idx = train_scores.index(i)
        #             score = train_scores.pop(score_idx)
                    essay = trainE.pop(score_idx)
    #                 print("essay", essay)
        #             sent_size = sent_size_list.pop(score_idx)
                    memory.append(essay)

        def test_step(e, m):
            feed_dict = {
                query: e,
                memory_key: m,
                keep_prob: 1
                #model.w_placeholder: word2vec
            }
            preds = sess.run(output, feed_dict)
            if is_regression:
                preds = np.clip(np.round(preds), min_score, max_score)
                return preds
            else:
                return preds

        saver = tf.train.import_meta_graph("runs/essay_set_7_cv_1_Jun_05_2020_02:19:41/checkpoints-15990.meta")  
        with tf.Session() as sess:
            saver.restore(sess,"runs/essay_set_7_cv_1_Jun_05_2020_02:19:41/checkpoints-15990")
            query = graph.get_tensor_by_name("input/question:0")
            memory_key = graph.get_tensor_by_name("input/memory_key:0")
            keep_prob = graph.get_tensor_by_name("input/keep_prob:0")
        #     for op in graph.get_operations():
        #         print(op.name)
            output = graph.get_tensor_by_name("prediction/predict_op:0")
    #         y0 = sess.run([y_])
        #     output=tf.get_collection('predict_op:0')

            test_preds = []
            for start in range(0, n_test, test_batch_size):
    #             print("E1")
                end = min(n_test, start+test_batch_size)
                print("Start: ", start, "End: ", end)
    #             print("Memory", memory)
                batched_memory = [memory] * (end-start)
    #             print("Batched_memory", batched_memory)
    #             print("Query: ", testE[start:end])
    #             print("E2")
    #             print(testE[start:end])
                query_list = []
                for elem2 in testE[start:end]:
    #                 print(len(elem2))
                    if (len(elem2) == max_sent_size):
                        query_list.append(elem2)
                    else:
                        elem2 = elem2[:max_sent_size]
                        query_list.append(elem2)
    #                     elem2 = [int(x) for x in elem2]
    #                     print(len(elem2))
    #                 else:
    #                     print(len(elem2))
    #                     elem2 = [int(x) for x in elem2]
                for ab in query_list:
                    print(len(ab))
    #             if (start == 15 and end == 30):
    #                 print("############################")
    #                 print(testE[15:30])
    #             for elem in batched_memory:
    #                 print(len(elem))
    #             print(batched_memory)
                preds = sess.run(output, feed_dict={query: query_list, memory_key:batched_memory, keep_prob:1})
                print("Preds", preds)
                print("type: ", type(preds))

                if is_regression:
                    preds = np.clip(np.round(preds), min_score, max_score)
                    print(preds)
    #                 return preds
    #             else:
    #                 continue
        #         preds = test_step(testE[start:end], batched_memory)
        #         print("Preds", preds)
        #         preds = preds.tolist()
                predsF = preds.astype('float32') 
                if type(predsF) is np.float32:
                    print("Here1")
                    test_preds = np.append(test_preds, predsF)
                else:
                    preds = preds.astype('int32')
                    print("Here2")
    #                 preds2 = []
    #                 for i in range(0, len(preds)):    
    #                     print(preds[i])
    #                     preds2.append(preds[i])
                    try:
                        preds2 = preds.tolist()
                    except ValueError:
                        print(testcase, e)
                        pass
    #                     preds2 = []
    #                     for i in range(0, len(preds)):    
    #                         print(preds[i])
    #                         pred2.append(preds[i])
                    print("type here: ", type(preds))
                    preds2 = preds.tolist()
                    print("Preds2",preds2)
    #                 print("pred_list", pred_list)
                    for ite in range(len(preds2)):
    #                     print("Done")
        #                 ite2 = ite.astype(numpy.int32)
        #                 print("Ite", type(ite))
        #                 print("pred ite", preds2[ite])
                        test_preds = np.append(test_preds, preds2[ite])
        #                 np.concatenate(test_preds, preds2[ite])
        #                 test_preds.append(preds2[ite])
    #             if not is_regression:
        #             test_preds = np.add(test_preds, min_score)
                    #test_kappp_score = kappa(test_scores, test_preds, 'quadratic')
        #             test_kappp_score = quadratic_weighted_kappa(test_scores, test_preds, min_score, max_score)
        #             print(test_kappp_score)
        #             stat_dict = {'pred_score': test_preds}
    #             stat_dict = {'essay_id': test_essay_id, 'org_score': test_scores, 'pred_score': test_preds}
                pred_dict = {'pred_score':test_preds}

    #         test_kappp_score = quadratic_weighted_kappa(test_scores, test_preds, min_score, max_score)
    #         print(test_kappp_score)
    #         stat_df = pd.DataFrame(stat_dict)
            pred_df = pd.DataFrame(pred_dict)
    #         print(stat_df)
    #         stat_df.to_csv('predicted_scores/prompt1/stats_'+testcase, index=None)
            pred_df.to_csv("predicted_scores/"+percent+"/prompt7/pred_"+testcase, index=None)


1569
Max Score 30
Min Score 0
==> glove is loaded
vocab size 1917495
train max sent size:  659
Number of files 39
['entities_beg_bound_7.csv', 'entities_beg_unbound_7.csv', 'entities_end_bound_7.csv', 'entities_end_unbound_7.csv', 'entities_mid_bound_7.csv', 'entities_mid_unbound_7.csv', 'remove_test_prompt_7.csv', 'repeat_test_conc1_prompt_7.csv', 'repeat_test_conc2_prompt_7.csv', 'repeat_test_into1_prompt_7.csv', 'repeat_test_into2_prompt_7.csv', 'repeat_test_middle1_prompt_7.csv', 'repeat_test_middle2_prompt_7.csv', 'repeat_test_middle3_prompt_7.csv', 'shuffle_test_prompt_7.csv', 'songs_test_beg_prompt_7.csv', 'songs_test_end_prompt_7.csv', 'songs_test_mid_prompt_7.csv', 'speeches_test_beg_prompt_7.csv', 'speeches_test_end_prompt_7.csv', 'speeches_test_mid_prompt_7.csv', 'uf_beg_bound_7.csv', 'uf_beg_unbound_7.csv', 'uf_end_bound_7.csv', 'uf_end_unbound_7.csv', 'uf_mid_bound_7.csv', 'uf_mid_unbound_7.csv', 'ut_beg_bound_7.csv', 'ut_beg_unbound_7.csv', 'ut_end_bound_7.csv', 'ut_end_u

Preds [ 6  8 16 16 16 13 13 17 17 13 13 13 16  6 16]
type:  <class 'numpy.ndarray'>
Here2
type here:  <class 'numpy.ndarray'>
Preds2 [6, 8, 16, 16, 16, 13, 13, 17, 17, 13, 13, 13, 16, 6, 16]
Start:  105 End:  120
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
Preds [ 8 16  8 16 13 13  8 13 13 17 13 16  8 13  8]
type:  <class 'numpy.ndarray'>
Here2
type here:  <class 'numpy.ndarray'>
Preds2 [8, 16, 8, 16, 13, 13, 8, 13, 13, 17, 13, 16, 8, 13, 8]
Start:  120 End:  135
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
Preds [13 13 16 16  8 13 13 13  8  6 16 16 16 16 16]
type:  <class 'numpy.ndarray'>
Here2
type here:  <class 'numpy.ndarray'>
Preds2 [13, 13, 16, 16, 8, 13, 13, 13, 8, 6, 16, 16, 16, 16, 16]
Start:  135 End:  150
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
Preds [17 16 16  8 13 16 17 23 16 16 13 13 16  6 16]
type:  <class 'numpy.ndarray'>
Here2
type here:  <class 'numpy.ndarray'>
Preds2 [17, 16, 16, 8, 13, 16, 17, 23, 16, 16, 13, 13, 16

Preds [13 16 16  8  6 16  8 13 16 16  8  8 16 16 13]
type:  <class 'numpy.ndarray'>
Here2
type here:  <class 'numpy.ndarray'>
Preds2 [13, 16, 16, 8, 6, 16, 8, 13, 16, 16, 8, 8, 16, 16, 13]
Start:  300 End:  314
659
659
659
659
659
659
659
659
659
659
659
659
659
659
Preds [13  8 16 13  8 16 13 14 17 16 23  6 13 13]
type:  <class 'numpy.ndarray'>
Here2
type here:  <class 'numpy.ndarray'>
Preds2 [13, 8, 16, 13, 8, 16, 13, 14, 17, 16, 23, 6, 13, 13]
Testcase:  entities_end_unbound_7.csv
314
training_data:  1569
Testcase:  entities_end_unbound_7.csv 314
Start:  0 End:  15
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
Preds [16 16 13 16 16 16 16 16 13 16 16 16  8  8 16]
type:  <class 'numpy.ndarray'>
Here2
type here:  <class 'numpy.ndarray'>
Preds2 [16, 16, 13, 16, 16, 16, 16, 16, 13, 16, 16, 16, 8, 8, 16]
Start:  15 End:  30
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
Preds [16 13 13 13  8  8 17 16  8 18 16  8 17 16 17]
type:  <class 'numpy.ndarray'>
Here2
typ

Preds [13 13 16 16  6 13 13 13  8  6 16 16 16 17 16]
type:  <class 'numpy.ndarray'>
Here2
type here:  <class 'numpy.ndarray'>
Preds2 [13, 13, 16, 16, 6, 13, 13, 13, 8, 6, 16, 16, 16, 17, 16]
Start:  135 End:  150
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
Preds [16 16 16  8 13 16 17 23 16 13 13 13 16  6 16]
type:  <class 'numpy.ndarray'>
Here2
type here:  <class 'numpy.ndarray'>
Preds2 [16, 16, 16, 8, 13, 16, 17, 23, 16, 13, 13, 13, 16, 6, 16]
Start:  150 End:  165
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
Preds [17 17  8 13 18 16 16 13 16  8 16  8  6  6 16]
type:  <class 'numpy.ndarray'>
Here2
type here:  <class 'numpy.ndarray'>
Preds2 [17, 17, 8, 13, 18, 16, 16, 13, 16, 8, 16, 8, 6, 6, 16]
Start:  165 End:  180
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
Preds [16 13 13 13 18  6 13  8  6 16 16 13 16 16 13]
type:  <class 'numpy.ndarray'>
Here2
type here:  <class 'numpy.ndarray'>
Preds2 [16, 13, 13, 13, 18, 6, 13, 8, 6, 16, 16, 13, 16,

Preds [13 16 16  8  8 16 13 16 17 16 13 13 16 16 13]
type:  <class 'numpy.ndarray'>
Here2
type here:  <class 'numpy.ndarray'>
Preds2 [13, 16, 16, 8, 8, 16, 13, 16, 17, 16, 13, 13, 16, 16, 13]
Start:  300 End:  314
659
659
659
659
659
659
659
659
659
659
659
659
659
659
Preds [13 13 16 13 13 16 13 16 17 16 23  6 13 13]
type:  <class 'numpy.ndarray'>
Here2
type here:  <class 'numpy.ndarray'>
Preds2 [13, 13, 16, 13, 13, 16, 13, 16, 17, 16, 23, 6, 13, 13]
Testcase:  remove_test_prompt_7.csv
314
training_data:  1569
Testcase:  remove_test_prompt_7.csv 314
Start:  0 End:  15
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
Preds [16 16 13 13 16 16 16 16 13 16 16 13  8  8 14]
type:  <class 'numpy.ndarray'>
Here2
type here:  <class 'numpy.ndarray'>
Preds2 [16, 16, 13, 13, 16, 16, 16, 16, 13, 16, 16, 13, 8, 8, 14]
Start:  15 End:  30
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
Preds [13  8 13 13  8  8 16 13  6 17 16  6 17 13  8]
type:  <class 'numpy.ndarray'>
Here2
ty

...

Testcase:  ut_mid_unbound_7.csv
314
training_data:  1569
Testcase:  ut_mid_unbound_7.csv 314
Start:  0 End:  15
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
Preds [17 16 16 17 18 17 17 17 13 16 16 17  8 13 16]
type:  <class 'numpy.ndarray'>
Here2
type here:  <class 'numpy.ndarray'>
Preds2 [17, 16, 16, 17, 18, 17, 17, 17, 13, 16, 16, 17, 8, 13, 16]
Start:  15 End:  30
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
Preds [16 16 13 16 13 13 17 16 13 23 16 13 17 16 17]
type:  <class 'numpy.ndarray'>
Here2
type here:  <class 'numpy.ndarray'>
Preds2 [16, 16, 13, 16, 13, 13, 17, 16, 13, 23, 16, 13, 17, 16, 17]
Start:  30 End:  45
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
Preds [16 16 13 17 16 16 23  8 16 17 16 16 16 17 13]
type:  <class 'numpy.ndarray'>
Here2
type here:  <class 'numpy.ndarray'>
Preds2 [16, 16, 13, 17, 16, 16, 23, 8, 16, 17, 16, 16, 16, 17, 13]
Start:  45 End:  60
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
Preds [1

Preds [ 6 17 16 13 16  6  8  8 16 17 13  8 16  6  6]
type:  <class 'numpy.ndarray'>
Here2
type here:  <class 'numpy.ndarray'>
Preds2 [6, 17, 16, 13, 16, 6, 8, 8, 16, 17, 13, 8, 16, 6, 6]
Start:  225 End:  240
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
Preds [ 6 16 13 13 16 13 13  6 13  6  8 16  8 13 23]
type:  <class 'numpy.ndarray'>
Here2
type here:  <class 'numpy.ndarray'>
Preds2 [6, 16, 13, 13, 16, 13, 13, 6, 13, 6, 8, 16, 8, 13, 23]
Start:  240 End:  255
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
Preds [ 8 13  8 13 13 13 16 16 17  8 13 16  8  6 16]
type:  <class 'numpy.ndarray'>
Here2
type here:  <class 'numpy.ndarray'>
Preds2 [8, 13, 8, 13, 13, 13, 16, 16, 17, 8, 13, 16, 8, 6, 16]
Start:  255 End:  270
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
Preds [16 17 13  8  8  8 17 16 16 13 17 16 17 13 16]
type:  <class 'numpy.ndarray'>
Here2
type here:  <class 'numpy.ndarray'>
Preds2 [16, 17, 13, 8, 8, 8, 17, 16, 16, 13, 17, 16, 17, 13, 16

Preds [13 13 16 16  6 19 13 13  8  8 16 17 16 16 13]
type:  <class 'numpy.ndarray'>
Here2
type here:  <class 'numpy.ndarray'>
Preds2 [13, 13, 16, 16, 6, 19, 13, 13, 8, 8, 16, 17, 16, 16, 13]
Start:  135 End:  150
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
Preds [16 16 16  8 12 16 17 23 16 16 13 13 16  6 16]
type:  <class 'numpy.ndarray'>
Here2
type here:  <class 'numpy.ndarray'>
Preds2 [16, 16, 16, 8, 12, 16, 17, 23, 16, 16, 13, 13, 16, 6, 16]
Start:  150 End:  165
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
Preds [17 17  6  8 17 16 13 14 16  6 16  8  6  6 16]
type:  <class 'numpy.ndarray'>
Here2
type here:  <class 'numpy.ndarray'>
Preds2 [17, 17, 6, 8, 17, 16, 13, 14, 16, 6, 16, 8, 6, 6, 16]
Start:  165 End:  180
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
Preds [16 13  8 13 17  8  8  8  6 16 16  6 16 17 13]
type:  <class 'numpy.ndarray'>
Here2
type here:  <class 'numpy.ndarray'>
Preds2 [16, 13, 8, 13, 17, 8, 8, 8, 6, 16, 16, 6, 16, 17,

Testcase:  wiki_mid_bound_7.csv
314
training_data:  1569
Testcase:  wiki_mid_bound_7.csv 314
Start:  0 End:  15
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
Preds [17 16 16 17 17 16 16 16 13 16 16 16  8 13 16]
type:  <class 'numpy.ndarray'>
Here2
type here:  <class 'numpy.ndarray'>
Preds2 [17, 16, 16, 17, 17, 16, 16, 16, 13, 16, 16, 16, 8, 13, 16]
Start:  15 End:  30
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
Preds [16 16 13 16 13  8 17 16  8 23 16  8 17 16 17]
type:  <class 'numpy.ndarray'>
Here2
type here:  <class 'numpy.ndarray'>
Preds2 [16, 16, 13, 16, 13, 8, 17, 16, 8, 23, 16, 8, 17, 16, 17]
Start:  30 End:  45
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
Preds [16 13 13 13 13 16 23 13 16 17 16 16 17 16 13]
type:  <class 'numpy.ndarray'>
Here2
type here:  <class 'numpy.ndarray'>
Preds2 [16, 13, 13, 13, 13, 16, 23, 13, 16, 17, 16, 16, 17, 16, 13]
Start:  45 End:  60
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
Preds [17 

659
659
659
659
659
659
659
659
659
659
659
659
Preds [16 16 16 16 18  6 13 13  8 16 16 13 16 17 16]
type:  <class 'numpy.ndarray'>
Here2
type here:  <class 'numpy.ndarray'>
Preds2 [16, 16, 16, 16, 18, 6, 13, 13, 8, 16, 16, 13, 16, 17, 16]
Start:  180 End:  195
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
Preds [ 6 13 16 13 16 24 13 16 16 16 23 13 13 13 17]
type:  <class 'numpy.ndarray'>
Here2
type here:  <class 'numpy.ndarray'>
Preds2 [6, 13, 16, 13, 16, 24, 13, 16, 16, 16, 23, 13, 13, 13, 17]
Start:  195 End:  210
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
Preds [17 13 16  8  8 24 13 17 17 13 16  6 24 17 13]
type:  <class 'numpy.ndarray'>
Here2
type here:  <class 'numpy.ndarray'>
Preds2 [17, 13, 16, 8, 8, 24, 13, 17, 17, 13, 16, 6, 24, 17, 13]
Start:  210 End:  225
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
Preds [ 6 17 16 17 16 16 13 13 16 17 13 13 16  6 16]
type:  <class 'numpy.ndarray'>
Here2
type here:  <class 'numpy.ndarray'>
Pred

In [4]:
import tensorflow as tf
import data_utils
from qwk import quadratic_weighted_kappa
import time
import os
import sys
import pandas as pd
import numpy as np

graph = tf.get_default_graph()

essay_set_id = 6
num_tokens = 42
embedding_size = 300
num_samples = 1
is_regression = True

early_stop_count = 0
max_step_count = 10
is_regression = False
gated_addressing = False
# essay_set_id = 1
batch_size = 15
embedding_size = 300
feature_size = 100
l2_lambda = 0.3
hops = 3
reader = 'bow' # gru may not work
epochs = 200
num_samples = 1
num_tokens = 42
test_batch_size = batch_size
random_state = 0

training_path = 'training_set_rel3.tsv'
essay_list, resolved_scores, essay_id = data_utils.load_training_data(training_path, essay_set_id)
print(len(resolved_scores))

max_score = max(resolved_scores)
min_score = min(resolved_scores)
if essay_set_id == 7:
    min_score, max_score = 0, 30
elif essay_set_id == 8:
    min_score, max_score = 0, 60

print("Max Score", max_score)
print("Min Score", min_score)
score_range = range(min_score, max_score+1)
# load glove
word_idx, word2vec = data_utils.load_glove(num_tokens, dim=embedding_size)

vocab_size = len(word_idx) + 1
print("vocab size", vocab_size)
# stat info on data set

sent_size_list = list(map(len, [essay for essay in essay_list]))
max_sent_size = max(sent_size_list)
print("train max sent size: ", max_sent_size)
mean_sent_size = int(np.mean(sent_size_list))
E = data_utils.vectorize_data(essay_list, word_idx, max_sent_size)

percentage = ["five", "ten", "fifteen", "twenty"]
for percent in percentage:
    testcases_path = "percent/"+percent+"/p6/"
    # print(testcases_path)
    testcases_files = os.listdir(testcases_path)
    print("Number of files", len(testcases_files))
    testcases_files = sorted(testcases_files)

    # te
    # testcases_files.remove("babel_prompt7.csv")
    # testcases_files.remove("test7.csv")
    # testcases_files.remove("svo_triplets_all_prompt7.csv")
    # testcases_files.remove("svo_triplets_random_prompt7.csv")
    # testcases_files.remove("entities_beg_unbound_1.csv")
    # testcases_files.remove("entities_end_bound_1.csv")
    # testcases_files.remove("entities_end_unbound_1.csv")
    # testcases_files.remove("entities_mid_bound_1.csv")
    # testcases_files.remove("entities_mid_unbound_1.csv")
    print(testcases_files)

    # testcases_files = ['disfluency_1.csv', 'incorrect_grammar_1.csv']
    for testcase in testcases_files:
        print("Testcase: ", testcase)
        testing_path = testcases_path+testcase
        apple = True
    #     if testcase == 'incorrect_1.csv':
    #         df_essay = pd.read_csv(testing_path, engine='python')
    # #         df_essay = df_essay[:0]
    #         df_score = pd.read_csv('statistics/pred_prompt1.csv')
    #         df_essay['rating'] = df_score['pred_score']
    #         df_essay['text2'] = df_essay['text']
    #         df_essay = df_essay[['text', 'rating']]
    #         print(df_essay.head())
    #     if (testcase =='test8.csv' or 'babel_prompt1.csv'):
    #         continue
    #         df_essay = pd.read_csv(testing_path, engine='python')
    #         df_essay = df_essay.iloc[:,0]
    #         print(df_essay.head())
        if apple==True:
    #         print("Before: ", df_essay)
            df_essay = pd.read_csv(testing_path, engine='python')
    #         print("Before: ", df_essay)
    #         df_essay[:-1]
    #         print("After: ", df_essay)
            print(len(df_essay))
    #         df_score = pd.read_csv('predicted_scores/org_scores/prompt2_org.csv')
    #         print(len(df_score))
    #         df_essay['rating'] = df_score['score']
    #         print(df_essay.head())

    #     testing_path = 'aes_data/essay1/fold_0/test.txt'
        essay_list_test, essay_id_test = data_utils.load_testcase_data(df_essay, essay_set_id)

    #     sent_size_list_test = list(map(len, [essay2 for essay2 in essay_list_test]))
    #     max_sent_size_test = max(sent_size_list_test)
    #     print("test max sent size: ", max_sent_size_test)
    #     mean_sent_size_test = int(np.mean(sent_size_list_test))

        E_test = data_utils.vectorize_data(essay_list_test, word_idx, max_sent_size)
    #     print(essay_list_test)
        test_batch_size = 15

        testE = []
        test_scores = []
        test_essay_id = []
        for test_index in range(len(essay_id_test)):
            testE.append(E_test[test_index])
    #         print(testE)
    #         test_scores.append(resolved_scores_test[test_index])
            test_essay_id.append(essay_id_test[test_index])

        trainE = []
        train_scores = []
        train_essay_id = []
        for train_index in range(len(essay_id)):
            trainE.append(E[train_index])
            train_scores.append(resolved_scores[train_index])
            train_essay_id.append(essay_id[train_index])

        n_train = len(trainE)    
        n_test = len(testE)
        print("training_data: ", n_train)
        print("Testcase: ", testcase, n_test)

        memory = []
        for i in score_range:
            for j in range(num_samples):
                if i in train_scores:
                    score_idx = train_scores.index(i)
        #             score = train_scores.pop(score_idx)
                    essay = trainE.pop(score_idx)
    #                 print("essay", essay)
        #             sent_size = sent_size_list.pop(score_idx)
                    memory.append(essay)

        def test_step(e, m):
            feed_dict = {
                query: e,
                memory_key: m,
                keep_prob: 1
                #model.w_placeholder: word2vec
            }
            preds = sess.run(output, feed_dict)
            if is_regression:
                preds = np.clip(np.round(preds), min_score, max_score)
                return preds
            else:
                return preds

        saver = tf.train.import_meta_graph("runs/essay_set_6_cv_1_Jun_05_2020_01:32:24/checkpoints-17575.meta")  
        with tf.Session() as sess:
            saver.restore(sess,"runs/essay_set_6_cv_1_Jun_05_2020_01:32:24/checkpoints-17575")
#         saver = tf.train.import_meta_graph("runs/essay_set_6_cv_1_Jun_05_2020_01:32:24/checkpoints-17575.meta")  
#         with tf.Session() as sess:
#             saver.restore(sess,"runs/essay_set_6_cv_1_Jun_05_2020_01:32:24/checkpoints-17575")
            query = graph.get_tensor_by_name("input/question:0")
            memory_key = graph.get_tensor_by_name("input/memory_key:0")
            keep_prob = graph.get_tensor_by_name("input/keep_prob:0")
        #     for op in graph.get_operations():
        #         print(op.name)
            output = graph.get_tensor_by_name("prediction/predict_op:0")
    #         y0 = sess.run([y_])
        #     output=tf.get_collection('predict_op:0')

            test_preds = []
            for start in range(0, n_test, test_batch_size):
    #             print("E1")
                end = min(n_test, start+test_batch_size)
                print("Start: ", start, "End: ", end)
    #             print("Memory", memory)
                batched_memory = [memory] * (end-start)
    #             print("Batched_memory", batched_memory)
    #             print("Query: ", testE[start:end])
    #             print("E2")
    #             print(testE[start:end])
                query_list = []
                for elem2 in testE[start:end]:
    #                 print(len(elem2))
                    if (len(elem2) == max_sent_size):
                        query_list.append(elem2)
                    else:
                        elem2 = elem2[:max_sent_size]
                        query_list.append(elem2)
    #                     elem2 = [int(x) for x in elem2]
    #                     print(len(elem2))
    #                 else:
    #                     print(len(elem2))
    #                     elem2 = [int(x) for x in elem2]
                for ab in query_list:
                    print(len(ab))
    #             if (start == 15 and end == 30):
    #                 print("############################")
    #                 print(testE[15:30])
    #             for elem in batched_memory:
    #                 print(len(elem))
    #             print(batched_memory)
                preds = sess.run(output, feed_dict={query: query_list, memory_key:batched_memory, keep_prob:1})
                print("Preds", preds)
                print("type: ", type(preds))

                if is_regression:
                    preds = np.clip(np.round(preds), min_score, max_score)
                    print(preds)
    #                 return preds
    #             else:
    #                 continue
        #         preds = test_step(testE[start:end], batched_memory)
        #         print("Preds", preds)
        #         preds = preds.tolist()
                predsF = preds.astype('float32') 
                if type(predsF) is np.float32:
                    print("Here1")
                    test_preds = np.append(test_preds, predsF)
                else:
                    preds = preds.astype('int32')
                    print("Here2")
    #                 preds2 = []
    #                 for i in range(0, len(preds)):    
    #                     print(preds[i])
    #                     preds2.append(preds[i])
                    try:
                        preds2 = preds.tolist()
                    except ValueError:
                        print(testcase, e)
                        pass
    #                     preds2 = []
    #                     for i in range(0, len(preds)):    
    #                         print(preds[i])
    #                         pred2.append(preds[i])
                    print("type here: ", type(preds))
                    preds2 = preds.tolist()
                    print("Preds2",preds2)
    #                 print("pred_list", pred_list)
                    for ite in range(len(preds2)):
    #                     print("Done")
        #                 ite2 = ite.astype(numpy.int32)
        #                 print("Ite", type(ite))
        #                 print("pred ite", preds2[ite])
                        test_preds = np.append(test_preds, preds2[ite])
        #                 np.concatenate(test_preds, preds2[ite])
        #                 test_preds.append(preds2[ite])
    #             if not is_regression:
        #             test_preds = np.add(test_preds, min_score)
                    #test_kappp_score = kappa(test_scores, test_preds, 'quadratic')
        #             test_kappp_score = quadratic_weighted_kappa(test_scores, test_preds, min_score, max_score)
        #             print(test_kappp_score)
        #             stat_dict = {'pred_score': test_preds}
    #             stat_dict = {'essay_id': test_essay_id, 'org_score': test_scores, 'pred_score': test_preds}
                pred_dict = {'pred_score':test_preds}

    #         test_kappp_score = quadratic_weighted_kappa(test_scores, test_preds, min_score, max_score)
    #         print(test_kappp_score)
    #         stat_df = pd.DataFrame(stat_dict)
            pred_df = pd.DataFrame(pred_dict)
    #         print(stat_df)
    #         stat_df.to_csv('predicted_scores/prompt1/stats_'+testcase, index=None)
            pred_df.to_csv("predicted_scores/"+percent+"/prompt6/pred_"+testcase, index=None)


1800
Max Score 4
Min Score 0
==> glove is loaded
vocab size 1917495
train max sent size:  489
Number of files 38
['entities_beg_bound_6.csv', 'entities_beg_unbound_6.csv', 'entities_end_bound_6.csv', 'entities_end_unbound_6.csv', 'entities_mid_bound_6.csv', 'entities_mid_unbound_6.csv', 'remove_test_prompt_6.csv', 'repeat_test_conc1_prompt_6.csv', 'repeat_test_conc2_prompt_6.csv', 'repeat_test_into1_prompt_6.csv', 'repeat_test_into2_prompt_6.csv', 'repeat_test_middle1_prompt_6.csv', 'repeat_test_middle2_prompt_6.csv', 'repeat_test_middle3_prompt_6.csv', 'songs_test_beg_prompt_6.csv', 'songs_test_end_prompt_6.csv', 'songs_test_mid_prompt_6.csv', 'speeches_test_beg_prompt_6.csv', 'speeches_test_end_prompt_6.csv', 'speeches_test_mid_prompt_6.csv', 'uf_beg_bound_6.csv', 'uf_beg_unbound_6.csv', 'uf_end_bound_6.csv', 'uf_end_unbound_6.csv', 'uf_mid_bound_6.csv', 'uf_mid_unbound_6.csv', 'ut_beg_bound_6.csv', 'ut_beg_unbound_6.csv', 'ut_end_bound_6.csv', 'ut_end_unbound_6.csv', 'ut_mid_bound_6

InvalidArgumentError: Restoring from checkpoint failed. This is most likely due to a mismatch between the current graph and the graph from the checkpoint. Please ensure that you have not altered the graph expected based on the checkpoint. Original error:

Assign requires shapes of both tensors to match. lhs shape= [31] rhs shape= [5]
	 [[node save/Assign_20 (defined at <ipython-input-1-604d438e395f>:171) ]]

Original stack trace for 'save/Assign_20':
  File "/home/rajivratn/anaconda3/envs/dependency/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/rajivratn/anaconda3/envs/dependency/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/rajivratn/anaconda3/envs/dependency/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/rajivratn/anaconda3/envs/dependency/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_i

In [5]:
import tensorflow as tf
import data_utils
from qwk import quadratic_weighted_kappa
import time
import os
import sys
import pandas as pd
import numpy as np

graph = tf.get_default_graph()

essay_set_id = 5
num_tokens = 42
embedding_size = 300
num_samples = 1
is_regression = True

early_stop_count = 0
max_step_count = 10
is_regression = False
gated_addressing = False
# essay_set_id = 1
batch_size = 15
embedding_size = 300
feature_size = 100
l2_lambda = 0.3
hops = 3
reader = 'bow' # gru may not work
epochs = 200
num_samples = 1
num_tokens = 42
test_batch_size = batch_size
random_state = 0

training_path = 'training_set_rel3.tsv'
essay_list, resolved_scores, essay_id = data_utils.load_training_data(training_path, essay_set_id)
print(len(resolved_scores))

max_score = max(resolved_scores)
min_score = min(resolved_scores)
if essay_set_id == 7:
    min_score, max_score = 0, 30
elif essay_set_id == 8:
    min_score, max_score = 0, 60

print("Max Score", max_score)
print("Min Score", min_score)
score_range = range(min_score, max_score+1)
# load glove
word_idx, word2vec = data_utils.load_glove(num_tokens, dim=embedding_size)

vocab_size = len(word_idx) + 1
print("vocab size", vocab_size)
# stat info on data set

sent_size_list = list(map(len, [essay for essay in essay_list]))
max_sent_size = max(sent_size_list)
print("train max sent size: ", max_sent_size)
mean_sent_size = int(np.mean(sent_size_list))
E = data_utils.vectorize_data(essay_list, word_idx, max_sent_size)

percentage = ["five", "ten", "fifteen", "twenty"]
for percent in percentage:
    testcases_path = "percent/"+percent+"/p5/"
    # print(testcases_path)
    testcases_files = os.listdir(testcases_path)
    print("Number of files", len(testcases_files))
    testcases_files = sorted(testcases_files)

    # te
    # testcases_files.remove("babel_prompt7.csv")
    # testcases_files.remove("test7.csv")
    # testcases_files.remove("svo_triplets_all_prompt7.csv")
    # testcases_files.remove("svo_triplets_random_prompt7.csv")
    # testcases_files.remove("entities_beg_unbound_1.csv")
    # testcases_files.remove("entities_end_bound_1.csv")
    # testcases_files.remove("entities_end_unbound_1.csv")
    # testcases_files.remove("entities_mid_bound_1.csv")
    # testcases_files.remove("entities_mid_unbound_1.csv")
    print(testcases_files)

    # testcases_files = ['disfluency_1.csv', 'incorrect_grammar_1.csv']
    for testcase in testcases_files:
        print("Testcase: ", testcase)
        testing_path = testcases_path+testcase
        apple = True
    #     if testcase == 'incorrect_1.csv':
    #         df_essay = pd.read_csv(testing_path, engine='python')
    # #         df_essay = df_essay[:0]
    #         df_score = pd.read_csv('statistics/pred_prompt1.csv')
    #         df_essay['rating'] = df_score['pred_score']
    #         df_essay['text2'] = df_essay['text']
    #         df_essay = df_essay[['text', 'rating']]
    #         print(df_essay.head())
    #     if (testcase =='test8.csv' or 'babel_prompt1.csv'):
    #         continue
    #         df_essay = pd.read_csv(testing_path, engine='python')
    #         df_essay = df_essay.iloc[:,0]
    #         print(df_essay.head())
        if apple==True:
    #         print("Before: ", df_essay)
            df_essay = pd.read_csv(testing_path, engine='python')
    #         print("Before: ", df_essay)
    #         df_essay[:-1]
    #         print("After: ", df_essay)
            print(len(df_essay))
    #         df_score = pd.read_csv('predicted_scores/org_scores/prompt2_org.csv')
    #         print(len(df_score))
    #         df_essay['rating'] = df_score['score']
    #         print(df_essay.head())

    #     testing_path = 'aes_data/essay1/fold_0/test.txt'
        essay_list_test, essay_id_test = data_utils.load_testcase_data(df_essay, essay_set_id)

    #     sent_size_list_test = list(map(len, [essay2 for essay2 in essay_list_test]))
    #     max_sent_size_test = max(sent_size_list_test)
    #     print("test max sent size: ", max_sent_size_test)
    #     mean_sent_size_test = int(np.mean(sent_size_list_test))

        E_test = data_utils.vectorize_data(essay_list_test, word_idx, max_sent_size)
    #     print(essay_list_test)
        test_batch_size = 15

        testE = []
        test_scores = []
        test_essay_id = []
        for test_index in range(len(essay_id_test)):
            testE.append(E_test[test_index])
    #         print(testE)
    #         test_scores.append(resolved_scores_test[test_index])
            test_essay_id.append(essay_id_test[test_index])

        trainE = []
        train_scores = []
        train_essay_id = []
        for train_index in range(len(essay_id)):
            trainE.append(E[train_index])
            train_scores.append(resolved_scores[train_index])
            train_essay_id.append(essay_id[train_index])

        n_train = len(trainE)    
        n_test = len(testE)
        print("training_data: ", n_train)
        print("Testcase: ", testcase, n_test)

        memory = []
        for i in score_range:
            for j in range(num_samples):
                if i in train_scores:
                    score_idx = train_scores.index(i)
        #             score = train_scores.pop(score_idx)
                    essay = trainE.pop(score_idx)
    #                 print("essay", essay)
        #             sent_size = sent_size_list.pop(score_idx)
                    memory.append(essay)

        def test_step(e, m):
            feed_dict = {
                query: e,
                memory_key: m,
                keep_prob: 1
                #model.w_placeholder: word2vec
            }
            preds = sess.run(output, feed_dict)
            if is_regression:
                preds = np.clip(np.round(preds), min_score, max_score)
                return preds
            else:
                return preds

        saver = tf.train.import_meta_graph("runs/essay_set_5_cv_1_Jun_05_2020_00:26:49/checkpoints-17100.meta")  
        with tf.Session() as sess:
            saver.restore(sess,"runs/essay_set_5_cv_1_Jun_05_2020_00:26:49/checkpoints-17100")
            query = graph.get_tensor_by_name("input/question:0")
            memory_key = graph.get_tensor_by_name("input/memory_key:0")
            keep_prob = graph.get_tensor_by_name("input/keep_prob:0")
        #     for op in graph.get_operations():
        #         print(op.name)
            output = graph.get_tensor_by_name("prediction/predict_op:0")
    #         y0 = sess.run([y_])
        #     output=tf.get_collection('predict_op:0')

            test_preds = []
            for start in range(0, n_test, test_batch_size):
    #             print("E1")
                end = min(n_test, start+test_batch_size)
                print("Start: ", start, "End: ", end)
    #             print("Memory", memory)
                batched_memory = [memory] * (end-start)
    #             print("Batched_memory", batched_memory)
    #             print("Query: ", testE[start:end])
    #             print("E2")
    #             print(testE[start:end])
                query_list = []
                for elem2 in testE[start:end]:
    #                 print(len(elem2))
                    if (len(elem2) == max_sent_size):
                        query_list.append(elem2)
                    else:
                        elem2 = elem2[:max_sent_size]
                        query_list.append(elem2)
    #                     elem2 = [int(x) for x in elem2]
    #                     print(len(elem2))
    #                 else:
    #                     print(len(elem2))
    #                     elem2 = [int(x) for x in elem2]
                for ab in query_list:
                    print(len(ab))
    #             if (start == 15 and end == 30):
    #                 print("############################")
    #                 print(testE[15:30])
    #             for elem in batched_memory:
    #                 print(len(elem))
    #             print(batched_memory)
                preds = sess.run(output, feed_dict={query: query_list, memory_key:batched_memory, keep_prob:1})
                print("Preds", preds)
                print("type: ", type(preds))

                if is_regression:
                    preds = np.clip(np.round(preds), min_score, max_score)
                    print(preds)
    #                 return preds
    #             else:
    #                 continue
        #         preds = test_step(testE[start:end], batched_memory)
        #         print("Preds", preds)
        #         preds = preds.tolist()
                predsF = preds.astype('float32') 
                if type(predsF) is np.float32:
                    print("Here1")
                    test_preds = np.append(test_preds, predsF)
                else:
                    preds = preds.astype('int32')
                    print("Here2")
    #                 preds2 = []
    #                 for i in range(0, len(preds)):    
    #                     print(preds[i])
    #                     preds2.append(preds[i])
                    try:
                        preds2 = preds.tolist()
                    except ValueError:
                        print(testcase, e)
                        pass
    #                     preds2 = []
    #                     for i in range(0, len(preds)):    
    #                         print(preds[i])
    #                         pred2.append(preds[i])
                    print("type here: ", type(preds))
                    preds2 = preds.tolist()
                    print("Preds2",preds2)
    #                 print("pred_list", pred_list)
                    for ite in range(len(preds2)):
    #                     print("Done")
        #                 ite2 = ite.astype(numpy.int32)
        #                 print("Ite", type(ite))
        #                 print("pred ite", preds2[ite])
                        test_preds = np.append(test_preds, preds2[ite])
        #                 np.concatenate(test_preds, preds2[ite])
        #                 test_preds.append(preds2[ite])
    #             if not is_regression:
        #             test_preds = np.add(test_preds, min_score)
                    #test_kappp_score = kappa(test_scores, test_preds, 'quadratic')
        #             test_kappp_score = quadratic_weighted_kappa(test_scores, test_preds, min_score, max_score)
        #             print(test_kappp_score)
        #             stat_dict = {'pred_score': test_preds}
    #             stat_dict = {'essay_id': test_essay_id, 'org_score': test_scores, 'pred_score': test_preds}
                pred_dict = {'pred_score':test_preds}

    #         test_kappp_score = quadratic_weighted_kappa(test_scores, test_preds, min_score, max_score)
    #         print(test_kappp_score)
    #         stat_df = pd.DataFrame(stat_dict)
            pred_df = pd.DataFrame(pred_dict)
    #         print(stat_df)
    #         stat_df.to_csv('predicted_scores/prompt1/stats_'+testcase, index=None)
            pred_df.to_csv("predicted_scores/"+percent+"/prompt5/pred_"+testcase, index=None)


1805
Max Score 4
Min Score 0
==> glove is loaded
vocab size 1917495
train max sent size:  452
Number of files 38
['entities_beg_bound_5.csv', 'entities_beg_unbound_5.csv', 'entities_end_bound_5.csv', 'entities_end_unbound_5.csv', 'entities_mid_bound_5.csv', 'entities_mid_unbound_5.csv', 'remove_test_prompt_5.csv', 'repeat_test_conc1_prompt_5.csv', 'repeat_test_conc2_prompt_5.csv', 'repeat_test_into1_prompt_5.csv', 'repeat_test_into2_prompt_5.csv', 'repeat_test_middle1_prompt_5.csv', 'repeat_test_middle2_prompt_5.csv', 'repeat_test_middle3_prompt_5.csv', 'songs_test_beg_prompt_5.csv', 'songs_test_end_prompt_5.csv', 'songs_test_mid_prompt_5.csv', 'speeches_test_beg_prompt_5.csv', 'speeches_test_end_prompt_5.csv', 'speeches_test_mid_prompt_5.csv', 'uf_beg_bound_5.csv', 'uf_beg_unbound_5.csv', 'uf_end_bound_5.csv', 'uf_end_unbound_5.csv', 'uf_mid_bound_5.csv', 'uf_mid_unbound_5.csv', 'ut_beg_bound_5.csv', 'ut_beg_unbound_5.csv', 'ut_end_bound_5.csv', 'ut_end_unbound_5.csv', 'ut_mid_bound_5

InvalidArgumentError: Restoring from checkpoint failed. This is most likely due to a mismatch between the current graph and the graph from the checkpoint. Please ensure that you have not altered the graph expected based on the checkpoint. Original error:

Assign requires shapes of both tensors to match. lhs shape= [31] rhs shape= [5]
	 [[node save/Assign_20 (defined at <ipython-input-1-604d438e395f>:171) ]]

Original stack trace for 'save/Assign_20':
  File "/home/rajivratn/anaconda3/envs/dependency/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/rajivratn/anaconda3/envs/dependency/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/rajivratn/anaconda3/envs/dependency/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/rajivratn/anaconda3/envs/dependency/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_i